In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo')
print(output.content)

Quantum mechanics is the branch of physics that describes the behavior of particles on a very small scale, where traditional laws of physics no longer apply and where particles can exist in multiple states simultaneously.


In [5]:
from langchain.schema import(
    SystemMessage,
    AIMessage,
    HumanMessage,
)

messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik ist die physikalische Theorie, die das Verhalten von Materie und Energie auf atomarer und subatomarer Ebene beschreibt.


## Caching LLM Responses
In-Memory Cache

In [6]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [7]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: user 336 ms, sys: 63.9 ms, total: 400 ms
Wall time: 1.28 s


"\n\nWhy did the banana go to the doctor?\n\nBecause it wasn't peeling well!"

In [8]:
%%time
llm.invoke(prompt)

CPU times: user 361 µs, sys: 0 ns, total: 361 µs
Wall time: 367 µs


"\n\nWhy did the banana go to the doctor?\n\nBecause it wasn't peeling well!"

## SQLite Caching

In [9]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

# Second request (cached, faster)
llm.invoke("Tell me a joke")

"\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired."

## LLM Streaming

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

Verse 1:
In the dead of night, under the pale moonlight
A raven takes flight, soaring through the sky
Its wings cut through the darkness, a shadow in the night
As it calls out to the moon, a haunting lullaby

Chorus:
Oh moon and raven, dancing in the night
A mysterious duo, a beautiful sight
Their bond unbreakable, their spirits untamed
Forever connected, in shadows they remain

Verse 2:
The moon shines down, casting an eerie glow
As the raven circles high, in the sky it roams
They share a connection, a silent understanding
A bond that transcends, the world they're standing in

Chorus:
Oh moon and raven, dancing in the night
A mysterious duo, a beautiful sight
Their bond unbreakable, their spirits untamed
Forever connected, in shadows they remain

Bridge:
The raven's call echoes, through the still night air
A symphony of darkness, a melody so rare
The moon watches on, with a silent gaze
As the raven flies on, in a mysterious haze

Chorus:
Oh moon and raven, dancing in the night
A myste

In [11]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Verse 1:
Under the moonlit sky, a raven takes flight
Guided by the darkness, soaring through the night
Its black feathers glisten in the pale moon's glow
A creature of mystery, in the shadows it roams

Chorus:
Moon and raven, dancing in the night
Their spirits intertwined, a haunting sight
In the silence of the darkness, their voices rise
A symphony of chaos, in the midnight skies
Verse 2:
The moon whispers secrets to the raven's ear
Tales of ancient wisdom, that only they can hear
They navigate the night, as one they roam
In harmony they sing, a melancholic tone

Chorus:
Moon and raven, dancing in the night
Their spirits intertwined, a haunting sight
In the silence of the darkness, their voices rise
A symphony of chaos, in the midnight skies
Bridge:
The moon shines bright, casting shadows on the land
The raven calls out, a cry that echoes through the sand
Together they are powerful, a force to be reckoned with
In the depths of the night, their magic is a gift

Chorus:
Moon and raven, 

## Prompt Templates

In [13]:
from langchain.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI
template = '''You are an experience virologist. Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='hiv', language='german')
prompt

'You are an experience virologist. Write a few sentences about the following virus "hiv" in german.'

In [14]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

HIV, das humane Immundefizienzvirus, ist ein Virus, das das Immunsystem des Menschen schwächt und zu AIDS führen kann. Es wird hauptsächlich durch ungeschützten Geschlechtsverkehr, den Austausch von infizierten Nadeln oder von der Mutter auf das Kind während der Schwangerschaft übertragen. Es ist wichtig, sich über die Übertragungswege und Präventionsmaßnahmen von HIV zu informieren, um die Verbreitung dieser lebensbedrohlichen Krankheit einzudämmen.


## ChatPromptTemplates

In [15]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

messages = chat_template.format_messages(n='10', area='Europe')
print(messages)

[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 10 countries in Europe by population.')]


In [16]:
# from langchain_openai import ChatOpenAI
output = llm.invoke(messages)
print(output.content)

{
    "1": "Germany",
    "2": "France",
    "3": "United Kingdom",
    "4": "Italy",
    "5": "Spain",
    "6": "Ukraine",
    "7": "Poland",
    "8": "Romania",
    "9": "Netherlands",
    "10": "Belgium"
}


## Simple Chains

In [18]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist. Write a few sentences about the following virus {virus} in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})

print(output)



> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist. Write a few sentences about the following virus HSV in Spanish.

> Finished chain.
{'virus': 'HSV', 'language': 'Spanish', 'text': 'El virus del herpes simple (HSV) es un virus común que afecta a millones de personas en todo el mundo. Hay dos tipos de HSV, HSV-1 que generalmente causa herpes labial y HSV-2 que generalmente causa herpes genital. El virus se transmite a través del contacto directo con las lesiones activas y puede permanecer latente en el cuerpo durante largos períodos de tiempo, causando brotes recurrentes. Aunque no hay cura para el HSV, existen tratamientos antivirales que pueden ayudar a controlar los síntomas y reducir la frecuencia de los brotes.'}


#### Another example with a single template value

In [19]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'

prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')
output = chain.invoke(country)
print(output['text'])



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of Türkiye?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of Türkiye is Ankara.

- Anıtkabir (Mausoleum of Atatürk)
- Kocatepe Mosque
- Ankara Citadel


## Sequential Chains

In [20]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
# FIRST LLM
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template = 'You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)

chain1 = LLMChain(
    llm=llm1,
    prompt=prompt_template1
)

# SECOND LLM
llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(
    llm=llm2,
    prompt=prompt_template2
)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

output = overall_chain.invoke('linear regression')



> Entering new SimpleSequentialChain chain...
Sure, here is a simple implementation of linear regression in Python:

```python
import numpy as np

def linear_regression(x, y):
    n = len(x)
    
    # Calculate the mean of x and y
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    
    # Calculate the slope (m) and y-intercept (b) of the regression line
    numer = 0
    denom = 0
    for i in range(n):
        numer += (x[i] - mean_x) * (y[i] - mean_y)
        denom += (x[i] - mean_x) ** 2
    m = numer / denom
    b = mean_y - m * mean_x
    
    return m, b

# Example usage
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 3, 4, 5, 6])

slope, intercept = linear_regression(x, y)
print("Slope:", slope)
print("Intercept:", intercept)
```

This function calculates the slope and y-intercept of the regression line that best fits the given data points (x, y). You can use these values to predict y for any given x value.
This Python function `linear_regression` calculates the coefficients 

In [21]:
print(output['output'])

This Python function `linear_regression` calculates the coefficients of a linear regression model based on input arrays `x` and `y`. Here is a step-by-step breakdown of how the function works:

1. Import the necessary libraries:
   - `numpy` as `np`: Numpy is a powerful library for numerical operations in Python.

2. Define the function `linear_regression` that takes two arguments `x` and `y`.

3. Initialize the variable `n` to store the length of input arrays `x` and `y`.

4. Calculate the mean of input arrays `x` and `y` using the `np.mean` function, storing the results in `mean_x` and `mean_y` variables, respectively.

5. Initialize variables `numer` and `denom` to zero, which will be used to calculate the slope (`m`) of the regression line.

6. Use a loop to iterate over each element in the input arrays `x` and `y`:
   - Update `numer` by adding `((xi - mean_x) * (yi - mean_y))` at each iteration.
   - Update `denom` by adding `((xi - mean_x) ** 2)` at each iteration.

7. Calculate

## LangChain Agents in Action: Python REPL

In [22]:
pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


## LangChain Tools: DuckDuckGp and Wikipedia

In [23]:
# pip install -q duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.invoke('Where was the Freddie Mercury born?')
print(output)

Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock 's most dynamic front men. Bulsara was born to Parsi parents ... But there are many lesser-known Freddie Mercury facts that reveal hidden depths to the life and work of the man born Farrokh Bulsara in Zanzibar, on September 5, 1946. Freddie Mercury was born Farrokh Bulsara in Stone Town, in the British protectorate of Zanzibar (now part of Tanzania), on September 5, 1946. His first big challenge was to come to terms with ... Freddie Mercury was Britain's first Indian rock star. This fact and the nature of his sexuality were the two areas of his short life about which he would be intentionally obscure. Although he was born on the East African island of Zanzibar on September 5, 1946, his parents were both Indian

In [25]:
search.name

'duckduckgo_search'

In [26]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [27]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
output = search.run('Freddie Mercury and Queen')
print(output)

[snippet: Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock's most dynamic front men.. Bulsara was born to Parsi parents who had emigrated from India to Zanzibar, where ..., title: Freddie Mercury | Biography, Parents, Songs, & Facts, link: https://www.britannica.com/biography/Freddie-Mercury], [snippet: Freddie Mercury always talked about his pride at writing "Killer Queen," which appeared on Queen's 1974 album Sheer Heart Attack. He said it was written in one night and was a song that he ..., title: Best Freddie Mercury Songs: 20 Essential Solo And Queen Tracks, link: https://www.udiscovermusic.com/stories/best-freddie-mercury-songs/], [snippet: Freddie Mercury Facts: 15 Things You Never Knew About The Queen Frontman 1: Freddie Mercury was a brilliant boxer

In [29]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region='de-de', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Berlin')

In [30]:
print(output)

[snippet: Berlin [bɛr'li:n] ist die Hauptstadt und ein Land der Bundesrepublik Deutschland. Die Großstadt ist mit rund 3,8 Millionen Einwohnern die bevölkerungsreichste und mit 892 Quadratkilometern die flächengrößte Gemeinde Deutschlands sowie die bevölkerungsreichste Stadt der Europäischen Union., title: Berlin - Wikipedia, link: https://de.wikipedia.org/wiki/Berlin], [snippet: Berlin-Magazin für Kultur, Gastro, Stadt, Veranstaltungen und Stil. Mit Empfehlungen für Kieze und Lebenslagen. Seit 1972. In Berlin, für Berlin. Für dich., title: tipBerlin - dein Berlin-Magazin, link: https://www.tip-berlin.de/], [snippet: Discover Berlin's culture, history, nightlife and nature with this guide to the top attractions. From Museum Island to the Berlin Wall, from clubs to parks, find out what makes the German capital so vibrant and diverse., title: 14 of the best things to do in Berlin - Lonely Planet, link: https://www.lonelyplanet.com/articles/best-things-to-do-in-berlin], [snippet: Einfach 

In [32]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)

for snipper, title, link in matches:
    print(f'Snippet: {snipper}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

Snippet: Berlin [bɛr'li:n] ist die Hauptstadt und ein Land der Bundesrepublik Deutschland. Die Großstadt ist mit rund 3,8 Millionen Einwohnern die bevölkerungsreichste und mit 892 Quadratkilometern die flächengrößte Gemeinde Deutschlands sowie die bevölkerungsreichste Stadt der Europäischen Union.
Title: Berlin - Wikipedia
Link: https://de.wikipedia.org/wiki/Berlin

--------------------------------------------------
Snippet: Berlin-Magazin für Kultur, Gastro, Stadt, Veranstaltungen und Stil. Mit Empfehlungen für Kieze und Lebenslagen. Seit 1972. In Berlin, für Berlin. Für dich.
Title: tipBerlin - dein Berlin-Magazin
Link: https://www.tip-berlin.de/

--------------------------------------------------
Snippet: Discover Berlin's culture, history, nightlife and nature with this guide to the top attractions. From Museum Island to the Berlin Wall, from clubs to parks, find out what makes the German capital so vibrant and diverse.
Title: 14 of the best things to do in Berlin - Lonely Planet
L

In [34]:
# pip install -q wikipedia

Note: you may need to restart the kernel to use updated packages.


In [35]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

In [37]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=5000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

'Page: Vector database\nSummary: A vector database management system (VDBMS) or simply vector database or vector store is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.\nVectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, and tens of thousands of dimensions might be used to represent sophisticated data. A vector\'s position in this space represents its characteristics. Words, phrases, or entire documents, and images, audio, and other types of data can all be vectorized.These feature vectors may be computed from the raw data using machine learning methods such as feature extraction algorithms, word embeddings or deep learning networks. The goal is 

In [38]:
wiki.invoke('Google Gemini')

'Page: Gemini (chatbot)\nSummary: Gemini, formerly known as Bard, is a generative artificial intelligence chatbot developed by Google. Based on the large language model (LLM) of the same name and developed as a direct response to the meteoric rise of OpenAI\'s ChatGPT, it was launched in a limited capacity in March 2023 before expanding to other countries in May. It was previously based on PaLM, and initially the LaMDA family of large language models.\nLaMDA had been developed and announced in 2021, but it was not released to the public out of an abundance of caution. OpenAI\'s launch of ChatGPT in November 2022 and its subsequent popularity caught Google executives off-guard and sent them into a panic, prompting a sweeping response in the ensuing months. After mobilizing its workforce, the company launched Bard in February 2023, which took center stage during the 2023 Google I/O keynote in May and was upgraded to the Gemini LLM in December. Bard and Duet AI were unified under the Gemi

## Creating a ReAct Agent

In [39]:
# pip install langchainhub -q

Note: you may need to restart the kernel to use updated packages.


In [45]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import AgentExecutor, Tool, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

template = '''
Answer the following questions as best you can.
Questions: {q}
'''

prompt_template = PromptTemplate.from_template(template=template)
prompt = hub.pull('hwchase17/react')
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you look up to a topic, country, or person on Wikipedia'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DucDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide'
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executer = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)


In [46]:
question = 'Generate the first 20 numbers in the Fibonacci series.'
output = agent_executer.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I can use Python to generate the Fibonacci series.
Action: Python REPL
Action Input: 
```python
def fibonacci(n):
    fib_series = [0, 1]
    for i in range(2, n):
        fib_series.append(fib_series[i-1] + fib_series[i-2])
    return fib_series

fibonacci(20)
```I now know the final answer
Final Answer: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]

> Finished chain.


In [47]:
print(output['input'])


Answer the following questions as best you can.
Questions: Generate the first 20 numbers in the Fibonacci series.


In [48]:
print(output['output'])

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]


In [50]:
question2 = 'Who is the current prime minister of the UK?'
output2 = agent_executer.invoke({
    'input': prompt_template.format(q=question2)
})



> Entering new AgentExecutor chain...
I should use DucDuckGo Search to find the current prime minister of the UK.
Action: DucDuckGo Search
Action Input: "current prime minister of the UK"The prime minister of the United Kingdom is the principal minister of the crown of His Majesty's Government, and the head of the British Cabinet.There is no specific date for when the office of prime minister first appeared, as the role was not created but rather evolved over a period of time through a merger of duties. The term was regularly, if informally, used of Robert Walpole by the 1730s. Rishi Sunak (born May 12, 1980, Southampton, England) British politician and financier who became leader of the Conservative Party and prime minster of the United Kingdom in October 2022. Previously he served as chancellor of the Exchequer (2020-22).. Early life. Sunak was born into a family with immigrant roots. His grandparents emigrated from Punjab, in northwestern India, to East Africa ... Boris Johnson, B

In [51]:
question3 = 'Tell me about Napolean Bonaparte early life'
output3 = agent_executer.invoke({
    'input': prompt_template.format(q=question3)
})



> Entering new AgentExecutor chain...
I should search for information about Napoleon Bonaparte's early life on Wikipedia.
Action: Wikipedia
Action Input: Napoleon Bonaparte early lifePage: Jérôme Bonaparte
Summary: Jérôme-Napoléon Bonaparte (born Girolamo Buonaparte; 15 November 1784 – 24 June 1860) was the youngest brother of Napoleon I and reigned as Jerome Napoleon I (formally Hieronymus Napoleon in German), King of Westphalia, between 1807 and 1813. 
From 1816 onward, he bore the title of Prince of Montfort. After 1848, when his nephew, Louis Napoleon, became President of the French Second Republic, he served in several official roles, including Marshal of France from 1850 onward, and President of the Senate in 1852. He was the only one of Napoleon's siblings who lived long enough to see the Bonaparte restoration.
Historian Owen Connelly points to his financial, military, and administrative successes and concludes he was a loyal, useful, and soldierly asset to Napoleon. Others, i

In [53]:
print(output3['input'])
print(output3['output'])


Answer the following questions as best you can.
Questions: Tell me about Napolean Bonaparte early life

Napoleon Bonaparte was born in Ajaccio, Corsica, on August 15, 1769. He began his education in Brienne, northern France, at the age of 10 and was admitted to the École Militaire in 1784 to train as an artillery officer.
